In [ ]:
import numpy as np
import json
from collections import Counter
from tqdm import tqdm
import nltk
import re
import string
from nltk.corpus import stopwords
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import warnings
from nltk.stem.snowball import SnowballStemmer
sb = SnowballStemmer("english")
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from nltk import pos_tag

def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc((s))))




def get_2d_spans(text, tokenss):
    spanss = []
    cur_idx = 0
    for tokens in tokenss:
        spans = []
        for token in tokens:
            if text.find(token, cur_idx) < 0:
                print(tokens)
                print("{} {} {}".format(token, cur_idx, text))
                raise Exception()
            cur_idx = text.find(token, cur_idx)
            spans.append((cur_idx, cur_idx + len(token)))
            cur_idx += len(token)
        spanss.append(spans)
    return spanss


def get_word_span(context, wordss, start, stop):
    spanss = get_2d_spans(context, wordss)
    idxs = []
    for sent_idx, spans in enumerate(spanss):
        for word_idx, span in enumerate(spans):
            if not (stop <= span[0] or start >= span[1]):
                idxs.append((sent_idx, word_idx))
    #print(spanss)
    #print(start,stop)
    #print(context[start:stop])
    assert len(idxs) > 0, "{} {} {} {}".format(context, spanss, start, stop)
    return idxs[0], (idxs[-1][0], idxs[-1][1] + 1)
def get_word_idx(context, wordss, idx):
    spanss = get_2d_spans(context, wordss)
    return spanss[idx[0]][idx[1]][0]



def process_tokens1(tokens):
   
    
    tokens = [w for w in tokens if w not in set(string.punctuation)]
   # stop_words = set(stopwords.words('english'))
    #tokens = [w for w in tokens if not w in stop_words]
    
    return tokens



def process_tokens(temp_tokens):
    tokens = []
    
    for token in temp_tokens:
        flag = False
        l = ("-", "\u2212", "\u2014", "\u2013", "/", "~", '"', "'", "\u201C", "\u2019", "\u201D", "\u2018", "\u00B0")
        # \u2013 is en-dash. Used for number to nubmer
        # l = ("-", "\u2212", "\u2014", "\u2013")
        # l = ("\u2013",)
        tokens.extend(re.split("([{}])".format("".join(l)), token))
    
    return tokens

def generatedateset(sentence_list,question_list):
    s_list=[]
    q_list=[]
    a_list=[]
    res=[]
    
    for q in question_list:
        if(len(q)==0):
            continue
        ai=q[0][0]
      
        pi=q[0][1]
        si=q[0][2]
        
        for i in range(len(sentence_list[ai][pi])):
            if(i==si):
                res+=[[q[0][3],sentence_list[ai][pi][i],q[0][4], q[0][5]]]
            else:
                res+=[[q[0][3],sentence_list[ai][pi][i],0, q[0][5]]]
    return res
def create_index_dict(embed,word_counter):
    word2idx={}
    idx2word={}
    i=1
    word2idx['<UNK>']=0
    idx2word[0]='<UNK>'
    vocab=np.array(['<UNK>'])
    for (key,value) in (embed.items()):
        word2idx[key]=i
        idx2word[i]=key
   #     vocab=np.append(vocab,[key])
        i+=1
    
    
    for key,value in (word_counter.items()):
        key=key.lower()
    
        try:
            embed[key]
           
        except: 
            try:
                word2idx[key]
            except:
                word2idx[key]=i
                idx2word[i]=key
        #        vocab=np.append(vocab,[key])
                i+=1
   
   # assert len(word2idx.keys())==len(vocab)
    return word2idx,idx2word,set(word2idx.keys())

def process(data):
    q_trn=[]
    s_trn=[]
    y_trn=[]
    q_id=[]
    for i in range(len(data)):
       # tokens = [w for w in tokens if w not in set(string.punctuation) ]
        
        
        question=list(filter(None, data[i][0]))#filter empty strin
        sentence=list(filter(None, data[i][1]))#filter empty strin
        index=data[i][2]
       # list1+=[[question,sentence,index]]
        
        q_trn+=[question]
        s_trn+=[sentence]
        y_trn+=[index]
        q_id+=[data[i][3]]
    return q_trn,s_trn,y_trn, q_id

def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=51):
            continue
        try:
            embed[(line[0])] = np.array(list(map(float, line[1:])))
        except:
            pass
    
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed

def load_data(data_type):
    source_path = "{}.json".format(data_type)
    source_data = json.load(open(source_path, 'r'))
    context_list=[]
    sentence_list=[]
    question_list=[]
    answer_list=[]
    answer_index=[]
    label_index=[]
    label=[]
    idxs=[]
    word_counter = Counter()


    for ai, article in enumerate(tqdm(source_data['data'])):
        s=[]
        p = []
        for pi, para in enumerate(article['paragraphs']):
            # wordss
            context = para['context']
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')

            si = list(map(word_tokenize, sent_tokenize(context)))
           # print(si)
            si = [process_tokens(tokens) for tokens in si]  # process tokens
            for sentence in si:
                for word in sentence:
                     word_counter[word] += 1

            # given xi, add chars
            s.append(si)
            p.append(context)

            indexi = [ai, pi]
            
            for qa in para['qas']:
                    qi = word_tokenize(qa['question'])
                    if qa['is_impossible']:
                        labeli=[0]
                      #  print(labeli)
                    else:
                        labeli=[1]       
                    yi = []
                    yyi=[] 
                    answers = []
                    q_id = qa['id']
                    
                    
                    ans=[]
                    if labeli[0] == 1:
                        ans=qa['answers']
                    else:
                        ans=qa['plausible_answers']
                    for answer in ans:
                        answer_text=answer['text']
                        answers.append(answer_text)
                        answer_start = answer['answer_start']
                        answer_stop = answer_start + len(answer_text)
                       # print(context)
                       # print(si)
                        yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                        #w0 = xi[yi0[0]][yi0[1]]            
                        #w1 = xi[yi1[0]][yi1[1]-1]
                        yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                       # print(yi)
                       # print(yi)
                        yyi.append([answer_start,answer_stop])
                    for qij in qi:

                                word_counter[qij] += 1
                    question_list.append(qi)
                    answer_list.append(answers)
                    answer_index.append(yi)
                    label.append(labeli)
                    label_index.append(indexi)
                   # ids.append(qa['id'])
                    idxs.append(len(idxs))

        sentence_list.append(s)

        context_list.append(p)
        
    
    a = {'word_counter':word_counter,'sentence_list':sentence_list,'answer_index':answer_index}
    with open("{}_saved.json".format(data_type), "w") as fp:
        json.dump(a , fp) 
    print("saved in json")
    return 
def load_from_json(data_type):
    with open("{}_saved2.json".format(data_type), "r") as fp:
            a=json.load(fp) 
  
    return a['word_counter'],a['context_list'],a['sentence_list'],a['answer_index']
def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=101):
            continue
        try:
            embed[(line[0])] = np.array(list(map(float, line[1:])))
        except:
            pass
    
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed

def make_output(output):
    if output == 0:
        return torch.tensor([0])
    else:
        return torch.tensor([1])
def create_emb_layer(weights_matrix, num_embeddings,trainable=False):
    _, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    
    
    #emb_layer.load_state_dict({'weight': weights_matrix})
    
    if not trainable:
        emb_layer.weight.requires_grad = False
    else:
        emb_layer.weight.requires_grad = True
    emb_layer.weight.data.copy_(weights_matrix)
    return emb_layer, embedding_dim



def evaluate(pred, test_set):
    """

    :param preds: A dictionary mapping question IDs to:
                    -> 0 if the question does not have a plausible answer
                    -> 1 if the question does have a plausible answer

    :param test_set: A dictionary of the following form:

    test_set["data"] -> list of titles
        title["title"] -> title name
        title["paragraphs"] -> list of paragraphs
            paragraph["context"] -> data which might answer questions in this paragraph
            paragraph["qas"] -> list of questions
                question["question"] -> question content
                question["id"] -> question ID

    :return: A 3-tuple consisting of (precision, recall, f1_score)
    """
    
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    n=len(pred)
    for i in range(n):
        if pred[i] == 0 and test_set[i] == 0:
            true_negatives += 1
        elif pred[i] == 1 and test_set[i]  == 1:
            true_positives += 1
        elif pred[i]== 0 and test_set[i] == 1:
            false_positives += 1
        elif pred[i] == 1 and test_set[i] == 0:
            false_negatives += 1

    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    acc=(true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    return precision, recall, f_1,acc


def calculate_precision(true_positives, false_positives):
    return true_positives / (true_positives + false_positives)


def calculate_recall(true_positives, false_negatives):
    return true_positives / (true_positives + false_negatives)


def calculate_f1(precision, recall):
    return (2 * precision * recall) / (precision + recall)


def get_we_slist(slist, word2idx):
    res_slist=[]
    for sent in slist:
        s=[]
        for word in sent:
            try:
                s.append(word2idx[word.lower()])
            except:
                s.append(word2idx['<UNK>'])
        res_slist.append(s)
                
    return res_slist
 
def create_prediction_file(preds, q_ids, fname):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    with open(fname, 'w') as outfile:
        json.dump(result, outfile)
    
def get_prediction_dict(preds, q_ids):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    return result
#create_embedding_matrix
def create_embeddingmatrix(vocab):
    
    initW = torch.nn.init.xavier_normal_(torch.randn([len(vocab), 300]))
    for i,v in enumerate(vocab):
        try:
            initW[i]=embed[v]
        except:
            continue
    embededing_matrix=initW
    return embededing_matrix


def generatedateset2(para_list,question_list):
   
    res=[]
    
    for q in question_list:
        if(len(q)==0):
            continue
        ai=q[0]
      
        pi=q[1]
        si=q[2]
        
        res+=[[para_list[ai][pi],' '.join(q[2]),0,q[3]]]
       
    return res

In [ ]:
embed_file="D:/glove.6B.100d.txt"
embed=read_embedding(embed_file)
print("finish loading embedding file")

In [ ]:
embed_dim=100

data_type='training'
word_counter,para_list,sentence_list,answer_index=load_from_json(data_type)
data_type='development'
d_word_counter,d_para_list,d_sentence_list,d_answer_index=load_from_json(data_type)


X=generatedateset2(para_list,answer_index)
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize

In [ ]:

def mapping(x):
    try:
        a=embed[x]
    except:
        a=embed['unk']
    
    return a

def sentence_tokenize(para):
    
    return sent_tokenize(para)

max_sentence=30
def get_avg_word_embedding(tokens,embed):
    
   
    s=np.array(list((map(lambda x: mapping(x.lower()),tokens))))
    s=np.mean(s,axis=0)
    return s
    
    
    
def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]


def mapping2(x,feature_names):
    try:
        return embed[feature_names[x]]
    except:
        return embed['unk']

def gettfidf_we(document,embed):
    
    tokenize = lambda doc: doc.lower().split(" ")
    document=[document]
    
    tf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    tfidf_matrix  = tf.fit_transform(document)

    feature_names = tf.get_feature_names()
    feature_index = tfidf_matrix [0,:].nonzero()[1]

    s=sum(tfidf_matrix[0, x] for x in feature_index)

    tfidf_scores = zip(feature_index, [tfidf_matrix[0, x]/s for x in feature_index])
    
 


    embedding=np.array(list(map(lambda x: mapping2(x,feature_names),feature_index)))

    scores=[]
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
          scores.append(s)
    scores=np.array(scores).reshape((len(scores),1))
    assert len(embedding)==len(scores)
    result=np.sum(np.multiply(embedding,scores),axis=0)
    return result


def cosineimilarity(e,v):
  
    #Input:
    #e = nxd input matrix with n row-vectors of dimensionality d (n is number of dictionary_keys)
    #v = mxd input matrix with m row-vectors of dimensionality d (m is number of test samples)
    # Output:
    # Matrix D of size nxm
    # s(i,j) is the cosinesimiarlity of embed(i,:) and test(j,:)
    g=e.dot(v.T)
    b=np.expand_dims(np.linalg.norm(e,axis=1),1)
    a=np.expand_dims(np.linalg.norm(v,axis=1),1)
    s=np.divide(g,np.multiply(b,a.T))
    # ... until here
    return s

def findknn(D,k):
    """
   # D=cos_distance matrix
   # k = number of nearest neighbors to be found
    
   # Output:
   # indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
   # dists = Euclidean distances to the respective nearest neighbors
    """
  
 
    m = D.shape[1]
    ind = np.argsort(D, axis=0)
    indices = ind[-k:]
    r = np.array([_ for _ in range(m)], dtype=np.int)
    r = np.array([r] * k)
    
    dists = D[indices, r]
    
    return indices, dists


def get_avg_word_sim(d,q):
   
    d_em=np.array(list((map(lambda x: mapping(x.lower()),d))))
    q_em=np.array(list((map(lambda x: mapping(x.lower()),q))))
    D=cosineimilarity(d_em,q_em)
    idx=findknn(D,1)[0].flatten()
    return np.mean(d_em[idx],axis=0)
def get_bleu(s,q):
    return sentence_bleu(s,q,weights=(1,0,0,0))
   
def getsem(l,sb):
  
    return [sb.stem(i) for i in l]


import re, math
from collections import Counter



def get_cos_similarity(s, q,max_length):
    
    sents=sentence_tokenize(s)
    cos_sim=np.zeros((max_length))
    q_vec=text_to_vector(q)
    for i,sent in enumerate(sents):
        if(i==max_length):
            break
        sent_vec=text_to_vector(sent)
        cos_sim[i]=get_cosine(sent_vec,q_vec)
        
    return cos_sim
        
        
def get_cosine(vec1, vec2):     
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     WORD = re.compile(r'\w+')
     words = WORD.findall(text)
     return Counter(words)
def get_ner_occur(q,para):
        NER_index={"NN":0,"VBD":1,"VB":2,"VBN":3,"JJ":4}
        NER_list=list(NER_index.keys())
       
       
        features=np.zeros(5)
      
        p=pos_tag(q)
   
        
        for i in range(len(p)):
            tag=p[i][1]


            word=p[i][0]
          # print(word)

            try:
                
                NER_index[tag]
           #     print(1)
                if(word not in para):
                    features[NER_index[tag]]+=1
            except:
                pass
  
        return features
def process(data,para_list,sentence_list,answer_index,mode='train'):
    NER_index={"NN":0,"VBD":1,"VB":2,"VBN":3,"JJ":4}
    NER_list=list(NER_index.keys())
    m=-1
    n=len(data)
    
    #vectorize
    para_total=[]
    for a in para_list:
        for j in a:
            para_total.append(j)

    vectorizer = CountVectorizer(stop_words='english',ngram_range=(1,1))
    corpus=para_total
    tt=vectorizer.fit_transform(corpus)
    bow_dim=tt.toarray().shape[1]
    max_sentence=30
    
    avg_para=np.zeros((n,embed_dim))
    avg_qst=np.zeros((n,embed_dim))
    weighted_para=np.zeros((n,embed_dim))
    weighted_qst=np.zeros((n,embed_dim))
    avg_word_sim=np.zeros((n,embed_dim))
    bleu=np.zeros((n,1))
    ner_occ=np.zeros((n,5))
  #  bow=np.zeros((n,bow_dim))
    
    cos_sim=np.zeros((n,max_sentence))
   # negation=np.zeros((n,1))
    for i in tqdm(range(n)):
        para=data[i][0]
        q=data[i][1]
        label=data[i][2]
        qid=data[i][3]
        #generate feature 
        #para=re.sub(r"[^A-Za-z1-9\-\"\.\,\?\!\s]",r"",para)
        
        para_tokens=list(filter(None,word_tokenize(para)))
        q_tokens=list(filter(None,word_tokenize(q)))
        avg_para[i]=get_avg_word_embedding(para_tokens,embed)
        avg_qst[i]=get_avg_word_embedding(q_tokens,embed)

        weighted_para[i]=gettfidf_we(para,embed)
        weighted_qst[i]=gettfidf_we(q,embed)
        
        avg_word_sim[i]=get_avg_word_sim(para_tokens,q_tokens)
        
        
        if(mode=='test'):
              
            ai=answer_index[i][0]
            pi=answer_index[i][1]
        else:
            ai=answer_index[i][0][0]
            pi=answer_index[i][0][1]
        
        sent_tokens=list(filter(None,sentence_list[ai][pi]))
        bleu[i]=get_bleu(sent_tokens,q_tokens)
        parsed_para= getsem(para_tokens,sb)
        parsed_q=getsem(q_tokens,sb)
        ner_occ[i]=get_ner_occur(parsed_q,parsed_para)
        cos_sim[i]=get_cos_similarity(" ".join(parsed_para)," ".join(parsed_q),max_sentence)
    res = np.column_stack((avg_word_sim,bleu,ner_occ,cos_sim,avg_para,avg_qst,weighted_para,weighted_qst))
    
    return res
    

In [10]:
X_dev=generatedateset2(d_para_list,d_answer_index)
feature=process(X,para_list,sentence_list,answer_index)

feature_d=process(X_dev,d_para_list,d_sentence_list,d_answer_index)

In [ ]:
#file1="D:/feature/feature_trn.npy"
#file2="D:/feature/feature_dev.npy"
#np.save(file1, feature)
#np.save(file2, feature_d)
#print("done")
#feature=np.load(file1)
#feature_d=np.load(file2)
#print("done")

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

s=StandardScaler(copy=True, with_mean=True, with_std=True)
s.fit(feature)
feature_norm=s.transform(feature)

y_trn=np.array([i[2] for i in X])
y_dev=np.array([i[2] for i in X_dev])
feature_d_norm=s.transform(feature_d)

In [47]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=6, max_depth=2, random_state=2)
rf.fit(feature_norm, y_trn)
rf.score(feature_norm,y_trn)
rf.score(feature_d_norm,y_dev)
pred=rf.predict(feature_d_norm)
print(evaluate(pred,y_dev))

(0.5466666666666666, 0.5677450161155545, 0.5570064999707209, 0.5652298850574713)


In [28]:

from sklearn.linear_model import LogisticRegression


# poly_lr = LogisticRegression(solver = 'lbfgs')
# poly_lr.fit(X_new ,trn_lbl)
# print(poly_lr.score(Xtest_new,tst_lbl))
a=0
t=540
lr = LogisticRegression(solver = 'lbfgs',C=1)
lr.fit(feature_norm[:,a:t] ,y_trn)
print(lr.score(feature_norm[:,a:t],y_trn))
print(lr.score(feature_d_norm[:,a:t],y_dev))
pred=lr.predict(feature_d_norm[:,a:t])
print(evaluate(pred,y_dev))

0.6276797517098683
0.6224137931034482
(0.6155172413793103, 0.6241258741258742, 0.6197916666666666, 0.6224137931034482)


In [16]:
#combine the dev and training 
from sklearn.preprocessing import StandardScaler

feature_sum=np.vstack((feature,feature_d))
feature_sum.shape
s=StandardScaler(copy=True, with_mean=True, with_std=True)
s.fit(feature_sum)
feature_sum_norm=s.transform(feature_sum)

lr = LogisticRegression(solver = 'lbfgs')
lr.fit(feature_sum_norm ,y_trn_sum)
print(lr.score(feature_sum_norm,y_trn_sum))
print('done')


0.6278564531702607
done


In [24]:
#load the test
data_type='test'
t_word_counter,t_para_list,t_sentence_list,t_question_index=load_from_json(data_type)
X_te=generatedateset2(t_para_list,t_question_index)




In [89]:
feature_te=process(X_te,t_para_list,t_sentence_list,t_question_index,'test')

100%|████████████████████████████████████| 11856/11856 [02:44<00:00, 71.99it/s]


In [90]:
#file3="D:/feature/feature_te.npy"
#np.save(file3, feature_te)
#file3="D:/feature/feature_te.npy"
#feature_te=np.load(file3)

In [27]:
#inference
feature_te_norm=s.transform(feature_te)
pred=lr.predict(feature_te_norm)
Id=[i[-1] for i in X_te]
result=list(zip(Id,pred))
import pandas as pd
labels = ['Id','Category']
df = pd.DataFrame.from_records(result, columns=labels)
df.to_csv("lr_fin1.csv",index=False,header=True,sep=",")
print('done')

done
